# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합-모듈화**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * Python 코드 모듈화
        * 각 모듈 코드 및 모델, 데이터파일을 일관성 있게 정리
        * .py 파일 생성 ==> 라이브러리 로딩, 각 task를 위한 함수 생성


## **1.환경설정**

* 경로 설정

구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6_2/'

## 2.모듈 구성하기

In [3]:
# %%writefile /content/drive/MyDrive/project6_2/emergency.py

# import os
# import requests
# import xml.etree.ElementTree as ET
# import pandas as pd
# import openai
# from openai import OpenAI
# import json
# import torch


# # 0. load key file------------------
# def load_key_file():



# # 1-1 audio2text--------------------



# # 1-2 text2summary------------------



# # 2. model prediction------------------



# # 3-1. get_distance------------------



# # 3-2. recommendation------------------




In [4]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
# %%writefile /content/drive/MyDrive/project6_2/emergency_onion.py

import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai

from haversine import haversine
from openai import OpenAI

import json
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

class miniproject6():
  def __init__(self, api_key=None, db_path=path+"응급실정보.csv"):
    self.c_id = None
    self.c_key = None
    self.model = None
    self.tokenizer = None

    if api_key:
      self.load_key_file(api_key)

    try:
      self.load_db_csv(db_path)
      # self.db = pd.read_csv(db_path)
    except:
      self.db = None
      print("응급실 데이터 필요합니다")

  def load_db_csv(self, db_path):
    self.db = pd.read_csv(db_path)

  # 0. load key file------------------
  def load_key_file(self, api_key=None, api_path=None):
    if api_key:
      openai.api_key = api_key
    elif api_path:
      with open(api_path, "r") as f:
        openai.api_key = f.readline().strip()
        os.environ['OPENAI_API_KEY'] = openai.api_key
    else:
      path = '/content/drive/MyDrive/project6_2/api_key.txt'

      if os.path.exists(path):
        self.load_key_file(api_path=path)
      else:
        raise ValueError("API_KEY 내놓으시오!")

  def load_maps_key_file(self, path=None):
    if not path:
      path = "/content/drive/MyDrive/project6_2/map_key.txt"
    with open(path, "r") as f:
      dict_string = f.readline()
      maps_client = json.loads(dict_string)

      self.c_id = maps_client["c_id"]
      self.c_key = maps_client["c_key"]

  # 1-1 audio2text--------------------
  def audio_to_text(self, audio_path):
    client = OpenAI()

    audio_file = open(audio_path, "rb")

    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="text"
    )

    return transcript

  # 1-2 text2summary------------------
  def text_to_summary(self, input_text, system_role=None):
    client = OpenAI()

    if not system_role:
      system_role = '''
      당신은 119 응급 전화를 받는 사람입니다.
      응급 전화를 받고 이 사람을 한국형 응급환자 분류도구 KTAS의 어느 등급에 분류할지 정리하기 위해 요약을 해야합니다.
      1. 진료 과목에 대한 분류가 필요합니다.
      2. 어떤 KTAS에 분류하기 전 분류 단계에 따라 병을 분류 해야합니다.
      3. KTAS 등급에 대해서는 요약하면 안됩니다.

      응답은 다음의 형식을 지켜주세요.
      {'입력텍스트':\'입력된 텍스트\', 'summary':\'텍스트 요약\'}
      '''

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    return response.choices[0].message.content

  # 2.0 load_model
  def load_bert_model(self, model_path):
    self.model = AutoModelForSequenceClassification.from_pretrained(model_path)

  # 2.01 load_tokenizer
  def load_tokenizer(self, tokenizer_path):
    self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

  # 2.02 train_model

  # 2. model prediction------------------
  def model_prediction(self, text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
      outputs = self.model(**inputs)

    logits = outputs.logits
    probabilities = logits.softmax(dim=1)

    pred = torch.argmax(probabilities, dim=-1).item()

    return pred, probabilities

  # 3-1. get_distance------------------
  def get_haversine_distance(self, lat1, long1, lat2, long2, unit="km"):
    return haversine((lat1, long1), (lat2, long2), unit=unit)

  def get_distance(self, lat1, long1, lat2, long2):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
      "X-NCP-APIGW-API-KEY-ID": self.c_id,
      "X-NCP-APIGW-API-KEY": self.c_key,
    }

    params = {
        "start": f"{long1},{lat1}",  # 출발지 (경도, 위도)
        "goal": f"{long2},{lat2}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(
        url,
        params=params,
        headers=headers
    )

    dist = response.json()['route']['trafast'][0]['summary']['distance']  # m(미터)

    # return response
    return dist

  # 3-2. recommendation------------------
  def recommend_hospital3(self, latitude, longitude, boundary=0.25):
    if latitude and longitude:
      df = self.db.copy()
      df = df[df["위도"].between(latitude-boundary, latitude+boundary)]
      df = df[df["경도"].between(longitude-boundary, longitude+boundary)]
      print(f'주변 응급실 총 개수: {len(df)}')

      df["haversine"] = df.apply(lambda x: self.get_haversine_distance(latitude, longitude, x["위도"], x["경도"]), axis=1)
      df = df.sort_values(by=["haversine"]).iloc[:10]

      df["distance"] = df.apply(lambda x: self.get_distance(latitude, longitude, x["위도"], x["경도"]), axis=1)
      df = df.sort_values(by=["distance"])

      for i in range(3):
        temp = df.iloc[i]

        print(f"병원 이름: {temp['병원이름']} // 주소: {temp['주소']} // 전화번호: {temp['전화번호 1']} // 응급실번호: {temp['전화번호 3']} // 거리: {int(temp['distance'])/1000}km")
    else:
      "본인 위치 기준"

In [6]:
# class 선언
project_class = miniproject6()
#project_class = project6_2()

# 필요한 key값 불러오기
project_class.load_key_file(api_path='/content/drive/MyDrive/project6_2/api_key.txt')
print(path)
print(path+'map_key.txt')
project_class.load_maps_key_file(path=path+'map_key.txt')

#project_class.load_key_file(api_path='/content/drive/MyDrive/project6_2/api_key.json')
#project_class.load_maps_key_file(path=path+'map_key.json')
# 모델 불러오기
#project_class.load_bert_model(path+"fine_tuned_bert_added_10/")
#project_class.load_tokenizer(path+"fine_tuned_bert_added_10/")
project_class.load_bert_model(path+"fine_tuned_bert/")
project_class.load_tokenizer(path+"fine_tuned_bert/")


# whisper 테스트
audio_text = project_class.audio_to_text(path+"/audio/audio2.mp3")
summary = eval(project_class.text_to_summary(audio_text))["summary"]

# prediction 테스트
emergency = project_class.model_prediction(summary)[0]

if emergency < 3:
  project_class.recommend_hospital3(37.358827, 127.114919)

/content/drive/MyDrive/project6_2/
/content/drive/MyDrive/project6_2/map_key.txt


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************RigA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
print(path)
print(path+'map_key.txt')